In [1]:
###############################################################################################
############################################ FUNCIONES ########################################
###############################################################################################

In [34]:
import numpy as np
import copy
import sys
from scipy.linalg import solve_triangular

In [2]:
def crea_matriz(renglones,columnas,maximo_valor,minimo_valor,entero=False):
    """
    Función de apoyo para genear matrices aleatorias
                            
    params: renglones       no. de renglones de la matriz
            columnas        no. de renglones de la matriz
            maximo_valor    valor máximo de las entradas de la matriz
            minimo_valor    valor mínimo de las entradas de la matriz
            entero          Indica si las entradas serán enteras (True) o no
            
    return: M               Matriz con numeros al azar
    """
    M=np.zeros((renglones, columnas))
    for i in range(renglones):
        for j in range(columnas):
            if entero:
                M[i][j]=(np.random.rand(1)*(maximo_valor+1-minimo_valor)+minimo_valor)//1
            else:
                M[i][j]=np.random.rand(1)*(maximo_valor-minimo_valor)+minimo_valor
    return M

In [3]:
def house(x):
    """
    Función que calcula la proyección de householder
    
    params: x       vector al que se le hará la reflexión householder
                    
    return: Beta    constante utilizada para obtener v
            v       vector que representa la reflexión de householder
    """
    m=len(x)
    norm_2_m=x[1:m].dot(np.transpose(x[1:m]))
    v=np.concatenate((1,x[1:m]), axis=None)
    Beta=0
    if (norm_2_m==0 and x[0]>=0):
        Beta=0
    elif (norm_2_m==0 and x[0]<0):
        Beta=2
    else:
        norm_x=np.sqrt(pow(x[0],2)+norm_2_m)
        if (x[0]<=0):
            v[0]=x[0]-norm_x
        else:
            v[0]=-norm_2_m/(x[0]+norm_x)
        Beta=2*pow(v[0],2)/(norm_2_m+pow(v[0],2))
        v=v/v[0]
    return Beta, v

In [4]:
def factorizacion_QR(A):
    """
    Función que genera una matriz que contendrá información escencial de las proyecciones householder
    (vectores v's) y componentes de la matriz triangular superior R, del estilo:
    [r11      r12      r13      r14    ]
    [v_2_(1)  r22      r23      r24    ]
    [v_3_(1)  v_3_(2)  r33      r34    ]
    [v_4_(1)  v_4_(2)  v_4_(3)  r44    ]
    [v_5_(1)  v_5_(2)  v_5_(3)  v_5_(4)]
    
    params: A      Matriz (mxn) de la que se desea obtner factorización QR
            
    return: A_r_v  Matriz (mxn) con la información escencial (es igual a la matriz R, pero en lugar de tener ceros
                   en la parte inferior, contiene info de los vectores householder que serán útiles para
                   futuros cálculos, que entre otros están el calcular la matriz ortonormal Q)

    """
    m=A.shape[0]
    n=A.shape[1]
    A_r_v=copy.copy(A)
    for j in range(n):
        beta, v=house(A_r_v[j:m,j])
        A_r_v[j:m,j:n]=A_r_v[j:m,j:n]-beta*(np.outer(v,v)@A_r_v[j:m,j:n])
        A_r_v[(j+1):m,j]=v[1:(m-j)]
    return A_r_v

In [5]:
def QT_C(A_r_v,C):
    """
    Función que calcula el producto matricial de Q_transpuesta por una matriz dada C
                            
    params: A_r_v   Matriz (mxn) con la info escencial
            C       Matriz (mxp) (si se pasa por ejemplo C=Identidad (mxm) la funcion devolverá Q)

    return: M       Matriz con numero al azar
    """
    m=A_r_v.shape[0]
    n=A_r_v.shape[1]
    QT_por_C=np.eye(m)
    for j in range(n-1,-1,-1):
        v=np.concatenate((1,A_r_v[(j+1):m,j]), axis=None)
        beta=2/(1+A_r_v[(j+1):m,j].dot(A_r_v[(j+1):m,j]))
        QT_por_C[j:m,j:m]=C[j:m,j:m]-beta*np.outer(v,v)@C[j:m,j:m]
    return QT_por_C

In [6]:
def Q_j(A_r_v,j):
    """
    Función que calcula la matriz Qj (en el proceso de obtención de factorización QR se van obteniendo n Qj's,
    que si se multiplican todas da por resultado Q=Q1*Q2*...*Qn)
                            
    params: A_r_v   Matriz (mxn) con la info escencial
            C       Matriz (mxp) (si se pasa por ejemplo C=Identidad (mxm) la funcion devolverá Q)

    return: Qj      Matriz Q de la j-esima iteración del proceso iterativo de factorización QR
    """
    m=A_r_v.shape[0]
    n=A_r_v.shape[1]
    Qj=np.eye(m)
    v=np.concatenate((1,A_r_v[(j+1):m,j]), axis=None)
    beta=2/(1+A_r_v[(j+1):m,j].dot(A_r_v[(j+1):m,j]))
    Qj[j:m,j:m]=np.eye(m-j)-beta*np.outer(v,v)
    return Qj

In [117]:
def Solucion_SEL_QR_nxn(A,b):
    """
    Función que obtiene la solución de un sistema de ecuaciones lineala (SEL) con n ecuaciones y n incognitas
            
    params: A   Matriz (nxn) que representa los coeficientas de las ecuaciones
            b   vector (nx1) constantes del sistema

    return: x   vector que satisface (Ax=b)
    """
    #FALTA CONTEMPLAR POSIBLES PROBLEMAS/SOLUCIONES PARA LOS DIFERENTES TIPOS DE SEL (SIN SOLUCION, MULTIPLES SOLUCIONES,
    #SOLUCION ÚNICA). INCORPORAR ESTO PARA EVITAR ERRORES FATALES/DESBORDAMIENTO/ETC
    A_r_v=factorizacion_QR(A)
    m=A_r_v.shape[0]
    n=A_r_v.shape[0]
    #Q=np.transpose(QT_C(A_r_v,np.eye(m)))
    #R=np.transpose(Q)@A
    Q=np.eye(m)
    R=copy.copy(A)
    for j in range(n):
        Qj=Q_j(A_r_v,j)
        Q=Q@Qj
        R=Q_j(A_r_v,j)@R
    b_prima=np.transpose(Q)@b
    #x=Solucion_SHA_triangula_nxn(R,b_prima)
    x = solve_triangular(R, np.transpose(Q)@b)
    return x

In [8]:
def Solucion_SHA_triangular_superior_nxn(R,b_prima):
    #ALGORITMO SIMPLE DE SUSTITUCION HACIA ATRAS CON UNA MATRIZ TRIANGULAR SUPERIOR (SOLO HAY QUE SUSTITUIR ITERATIVAMENTE)
    return x

In [9]:
###############################################################################################
##################################### PRUEBA DE FUNCIONES #####################################
###############################################################################################

In [10]:
m=5
n=3
A=np.round(crea_matriz(m,n,6,-6,False),2)

#Prueba también definiendo una matriz entrada por entrada, solo asegurate que sus entradas sean
#tipo flotantes/dobles (pues si son de tipo enteras arrastrarán errores de redondeo considerables)
# m=3
# n=3
#A=np.array([[1, 2, 1], [2, 3, 2], [1, 2, 3]], dtype='d')

In [11]:
A_r_v=factorizacion_QR(A)
np.round(A_r_v,4)

array([[ 6.6072,  0.6731, -5.6347],
       [-0.2181,  6.1281,  3.5372],
       [ 0.1827,  1.6667,  5.5272],
       [-0.2189, -0.2115,  5.074 ],
       [ 0.1942,  0.1549,  5.9971]])

In [12]:
#Obtención de Q con la función QT_C
Q=np.transpose(QT_C(A_r_v,np.eye(m)))
np.round(Q,4)

array([[-0.7144,  0.3738, -0.3133,  0.3754, -0.333 ],
       [ 0.3738,  0.9185,  0.0683, -0.0818,  0.0726],
       [-0.3133,  0.0683,  0.9427,  0.0686, -0.0608],
       [ 0.3754, -0.0818,  0.0686,  0.9178,  0.0729],
       [-0.333 ,  0.0726, -0.0608,  0.0729,  0.9353]])

In [13]:
#Así obtenemos R
R=np.transpose(Q)@A
np.round(R,4)

array([[ 6.6072,  0.6731, -5.6347],
       [-0.    ,  2.942 , -2.952 ],
       [ 0.    , -5.3104, -5.4648],
       [-0.    ,  0.6738,  0.4779],
       [ 0.    , -0.4937, -2.0626]])

In [14]:
#Ejemplo ilustrativo de cómo obtener Q y R, visualizando cada iteración
Q=np.eye(m)
R=copy.copy(A)
for j in range(n):
    Qj=Q_j(A_r_v,j)
    print('\nQ',j,':\n',np.round(Qj,4))
    Q=Q@Qj
    R=Q_j(A_r_v,j)@R
    print('\nAplicando las Qjs a A por la izquierda para obtener R, iteracion ',j,':\n',np.round(R,4))

print('\n\n\nResultados finales:')
print('\nR es el resultado de multiplicar todas las Qjs a A\n',np.round(R,4))
print('\nQ es el resultado de multiplicar todas las Qjs y transponer:\n',np.round(Q,4))



Q 0 :
 [[-0.7144  0.3738 -0.3133  0.3754 -0.333 ]
 [ 0.3738  0.9185  0.0683 -0.0818  0.0726]
 [-0.3133  0.0683  0.9427  0.0686 -0.0608]
 [ 0.3754 -0.0818  0.0686  0.9178  0.0729]
 [-0.333   0.0726 -0.0608  0.0729  0.9353]]

Aplicando las Qjs a A por la izquierda para obtener R, iteracion  0 :
 [[ 6.6072  0.6731 -5.6347]
 [-0.      2.942  -2.952 ]
 [ 0.     -5.3104 -5.4648]
 [-0.      0.6738  0.4779]
 [ 0.     -0.4937 -2.0626]]

Q 1 :
 [[ 1.      0.      0.      0.      0.    ]
 [ 0.      0.4801 -0.8666  0.1099 -0.0806]
 [ 0.     -0.8666 -0.4443  0.1833 -0.1343]
 [ 0.      0.1099  0.1833  0.9767  0.017 ]
 [ 0.     -0.0806 -0.1343  0.017   0.9875]]

Aplicando las Qjs a A por la izquierda para obtener R, iteracion  1 :
 [[ 6.6072  0.6731 -5.6347]
 [-0.      6.1281  3.5372]
 [ 0.      0.      5.3509]
 [-0.     -0.     -0.8944]
 [ 0.      0.     -1.0572]]

Q 2 :
 [[ 1.      0.      0.      0.      0.    ]
 [ 0.      1.      0.      0.      0.    ]
 [ 0.      0.      0.9681 -0.1618 -0.1913]

In [15]:
##################################### FUNCION DESECHADA #####################################

# def Factorizacion_Q_R(A):
#     m=A.shape[0]
#     n=A.shape[1]
#     R=copy.copy(A)
#     Q=np.eye(m)
#     for j in range(n):
#         Qj=np.eye(m)
#         beta, v=house(R[j:m,j])
#         P=np.eye(m-j)-beta*(np.outer(v,v))
#         R[j:m,j:n]=P@R[j:m,j:n]
#         Qj[j:m,j:m]=P
#         Q=Qj@Q
#     Q=np.transpose(Q)
#     return Q,R


# Q,R=Factorizacion_Q_R(A)
# print('\n\nQ:\n',np.round(Q,4))
# print('\nR:\n',np.round(R,4))
# print('\nQ@R:\n',np.round(Q@R,4))
# print('\nA:\n',A)

In [16]:
#Prubemos un ejemplo más grande con 10^4 entradas
m=125
n=80
A=np.round(crea_matriz(m,n,10,-10,False),2)
A_r_v=factorizacion_QR(A)
A_r_v

array([[ 6.57258458e+01, -9.02900515e+00, -5.51529000e+00, ...,
         3.68092639e+00,  5.04291875e+00, -4.67291667e+00],
       [ 1.06189872e-01,  6.28972660e+01, -1.71800761e+00, ...,
        -1.65263617e+00,  8.67220428e+00,  2.57170497e+00],
       [ 2.84800422e-03,  4.52402847e-02,  6.22779128e+01, ...,
         9.98059676e+00, -3.18247865e+00, -1.26995984e+01],
       ...,
       [ 9.71033820e-02, -9.83803022e-02,  2.61975378e-02, ...,
         2.24952874e-01,  2.13025970e-02,  1.30474492e-01],
       [-9.07292774e-02,  8.91852938e-02, -1.09253068e-01, ...,
         1.61016740e-01,  2.60233535e-02,  3.03451385e-02],
       [ 2.56320380e-02, -9.50611807e-03, -5.57806922e-02, ...,
        -5.19127394e-02,  3.90623445e-02,  1.05213578e-01]])

## Eliminación por bloques

In [118]:
def bloques(A, b=False, n1=False, n2=False):
    
    """
    Esta es la función para la creación de bloques usando un arreglo de numpy
    """

    # Primero definimos el n
    m,n = A.shape

    # Condiciones de A
    # Si no se dan los n deseados, se intentan hacer los bloques casi iguales
    if  not (n1&n2):
        n1 = n//2
        n2 = n - n1
    # Los bloques deben cumplir la condicion de tamaño
    elif n1+n1 != n:
        sys.exit('n1 + n2 debe ser igual a n')
    else:
        None

    # Condiciones de b
    if  b is False:
        b1 = None
        b2 = None
        print('condicion1')
    elif len(b) == m:
        b1 = b[:n1]
        b2 = b[n1:m]
    else:
        sys.exit('los renglones de A y b deben ser del mismo tamaño')


    A11 = A[:n1,:n1]
    A12 = A[:n1,n1:n]
    A21 = A[n1:m,:n1]
    A22 = A[n1:m,n1:n]

    return A11,A12,A21,A22,b1,b2

In [119]:
def eliminacion_bloques(A,b):

    ## Sean A y A11 no singulares

    if np.linalg.det(A)==0:
        sys.exit('A debe ser no singular')

    A11,A12,A21,A22,b1,b2 = bloques(A,b)

    if np.linalg.det(A11)==0:
        ys.exit('A11 debe ser no singular')

    ## 1. Calcular A11^{-1}A12 y A11^{-1}b1 teniendo cuidado en no calcular la inversa sino un sistema de ecuaciones lineales
    ## Aquí se debe usar el método QR una vez que esté desarrollado

    ## Definimos y = A11^{-1}b1, por tanto A11y=b1. Resolviendo el sistema anterior para 11y:
    y = Solucion_SEL_QR_nxn(A11,b1)
    #y = np.linalg.solve(A11,b1)

    ## Definimos Y = A11^{-1}A12
    Y = Solucion_SEL_QR_nxn(A11,A12)
    #Y = np.linalg.solve(A11,A12)

    ## 2. Calcular el complemento de Schur del bloque A11 en A. Calcular b_hat
    S = A22 - A21@Y
    b_h = b2 - A21@y

    ## 3. Resolver Sx2 = b_hat
    x2 = Solucion_SEL_QR_nxn(S,b_h)
    #x2 = np.linalg.solve(S,b_h)

    ## 4. Resolver A11x1 = b1-A12X2
    x1 = Solucion_SEL_QR_nxn(A11,b1-A12@x2)
    #x1 = np.linalg.solve(A11,b1-A12@x2)

    return np.concatenate((x1,x2), axis=0)

In [120]:
m=6
n=6
A=np.round(crea_matriz(m,n,6,-6,False),2)
b = np.round(crea_matriz(m,1,6,-6,False),2)

In [121]:
print("A:",A)
print("b:",b)

A: [[ 4.65  2.82  5.56  5.23  5.08 -1.02]
 [ 4.99  2.82 -1.12  4.15  0.87 -1.08]
 [ 1.42  1.    0.04 -5.8  -4.74 -1.62]
 [-1.98  5.03 -1.91 -3.91 -2.21  3.76]
 [-0.44  2.65  4.68  4.33 -2.33 -0.62]
 [-3.41  3.98  4.17  5.75 -3.51 -1.08]]
b: [[ 4.87]
 [ 2.19]
 [ 4.3 ]
 [-0.68]
 [-0.66]
 [ 5.38]]


Primero resolvamos el sistema de ecuaciones usando la paquetería de numpy para comparar.

In [126]:
np.linalg.solve(A,b)

array([[-1.36450007],
       [ 2.20883656],
       [-0.48675371],
       [-0.85081159],
       [ 1.57484597],
       [-4.06067058]])

Ahora usemos la factorización QR

In [127]:
Solucion_SEL_QR_nxn(A,b)

array([[-1.36450007],
       [ 2.20883656],
       [-0.48675371],
       [-0.85081159],
       [ 1.57484597],
       [-4.06067058]])

Por último usemos eliminación por bloques con QR

In [128]:
eliminacion_bloques(A,b)

array([[-1.36450007],
       [ 2.20883656],
       [-0.48675371],
       [-0.85081159],
       [ 1.57484597],
       [-4.06067058]])